In [ ]:
from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO

from pydantic import BaseModel
from enum import StrEnum

In [ ]:
client = genai.Client(api_key="xxxxxxxxxxxx")

In [ ]:
DEFAULT_MODEL_TEXT = "gemini-2.5-flash"
DEFAULT_MODEL_IMAGE = "imagen-4.0-generate-preview-06-06"

In [ ]:
DEFAULT_ASPECT_RATIO = "1:1"

In [ ]:
DEFAULT_THINKING = 0 # SET TO FALSE
# https://ai.google.dev/gemini-api/docs/thinking

In [ ]:
ILUSTRATIONS_STYLE = {
    "Handcrafted Knitted": "Simulates textures of knitting, crochet, or handcrafted felt.",
    "Fantasy Star": "Epic aesthetic of classic RPGs and fantasy book covers.",
    "LEGO": "Colorful plastic blocks with a modular appearance.",
    "Pixel Art": "Visual style of 8-bit and 16-bit games, with retro charm.",
    "DreamWorks": "Caricatured 3D style typical of movies like Shrek and Kung Fu Panda.",
    "Muppet": "Puppet-like visuals, with textures resembling plush and fabric.",
    "Attack on Titan": "Inspired by the anime Shingeki no Kyojin, with bold and dark lines.",
    "Fleischer Studios": "1930s style, like Betty Boop and Popeye.",
    "Minecraft": "Cubic block aesthetic typical of the Minecraft universe.",
    "Simpsons": "Visual style of the animated series, with yellow skin and exaggerated features.",
    "Vintage Comic": "Aesthetic of old comic books, like Marvel’s 1960s issues.",
    "Cartoon Network": "Visual style inspired by 2000s animations like Samurai Jack and others."
}

In [ ]:
class Language(StrEnum):
    ENGLISH = "English"
    SPANISH = "Spanish"
    PORTUGUESE = "Portuguese"
    FRENCH = "French"

In [ ]:
class Carrossel(BaseModel):
    title: str
    slide: list[str]
    legend: str

In [ ]:
input_message = "Mitos sobre o filme O Poderoso Chefão e fatos curiosos"

In [ ]:
DEFAULT_STYLE = "Vintage Comic: Aesthetic of old comic books, like Marvel’s 1960s issues."

In [ ]:
DEFAULT_N_SLIDES = 5

In [ ]:
language = (Language.PORTUGUESE)

In [ ]:
def get_from_gemini_text(input_message: str, sys_instructs:str, response_model=Carrossel, model=DEFAULT_MODEL_TEXT):
  response = client.models.generate_content(
    model=model,
    contents=input_message,
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=DEFAULT_THINKING),
        system_instruction=sys_instructs,
        response_mime_type="application/json",
        response_schema=response_model
    )
  )

  converted: response_model = response.parsed
  return converted

In [ ]:
def get_from_gemini_image(input_messge: str, model=DEFAULT_MODEL_IMAGE, n_image=DEFAULT_N_SLIDES, aspect_ratio=DEFAULT_ASPECT_RATIO):
  response = client.models.generate_images(
    model=model,
    prompt=input_messge,
    config=types.GenerateImagesConfig(
        number_of_images=n_image,
        aspectRatio=DEFAULT_ASPECT_RATIO
    )
  )
  return response

In [ ]:
system_instruction = f"""
  Use the provided message to create copy for a carousel post in {language}.

  Divide the content into {DEFAULT_N_SLIDES} slides:
  1. **Slide 1 (hook)**: headline or main call-to-action, 4 to 8 words, short and attention-grabbing.
  2. **Following slides**: each with 15 to 25 words, containing one key idea per slide.
    - Keep text concise.
    - Provide details and value.
  3. **Last Slide**: Share this post with someone who needs to see it!

  **Caption**: write a complementary text of 100 to 150 characters in {language}.
"""

In [ ]:
response = get_from_gemini_text(input_messge=input_message, sys_instructs=system_instruction)
print(response)

In [ ]:
def generate_images_from_class(carrossel: Carrossel, language: Language, style: str):
  prompt_imagem = f"""
    Create an illustration in the style of {style}.
    The text must be written in {language} and seamlessly integrated into the body of the image, ensuring it is clear and visually appealing.
    Include the following text exactly as provided:
  """

  slide_images = []
  for prompt in carrossel.slide:
    prmpt = prompt_imagem + prompt
    slide_images.append(get_from_gemini_image(input_messge=prmpt, n_image=1))

  return slide_images

In [ ]:
slide_images = generate_images_from_class(carrossel=response, language=language, style=DEFAULT_STYLE)

In [ ]:
for i in slide_images:
  for generated_image in i.generated_images:
    generated_image.image.show()